# Inaugural Project

> **Note the following:** 
> 1. This is an example of how to structure your **inaugural project**.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `inauguralproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [6]:
import numpy as np

# autoreload modules when code is run. Otherwise, python will not see recent changes. 
%load_ext autoreload
%autoreload 2

# Import your own code
import inauguralproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Det hele skal nok gå

In [7]:
# Baseline Parameters
# Preferences: 
ro = 2
ny = 0.001
epsilon = 1
omega = 0.5

# Household production: 
alpha = 0.5
sigma = 1

# Wages: 
omega_M = omega_F = 1


# Question 1

**Explain how you solve the model**

In [8]:
# code for solving the model (remember documentation and comments)

a = np.array([1,2,3])
b = inauguralproject.square(a)
print(b)

[1 4 9]


# Question 2

Explain your code and procedure

In [9]:
# code

# Question 3

Explain your code and procedure

In [10]:
# code

ADD CONCISE CONLUSION.